In [1]:
import os
import re
import json
import pandas as pd

def generar_tablas_estaciones(directorio_estaciones, logger=None, view=None):
  # directorio_estaciones: directorio donde se encuentran los archivos JSON
  # logger: (true) vuelca por pantalla la secuencia de ficheros leídos
  # Diccionario para almacenar los DataFrames por estación
  # Cada clave será el código de la estación (ej. 'C302')
  # Cada valor será un DataFrame con 'Tiempo', 'Estación' y columnas para cada sustancia
  matrices_tiempo_por_estacion = {}

  # Patrón para extraer el código de la estación y la sustancia del nombre del archivo
  patron_nombre_archivo = re.compile(r"^(C\d{3})_(.+)\.json$")

  # Obtener la lista de archivos y ordenarla
  lista_archivos = os.listdir(directorio_estaciones)
  lista_archivos.sort()

  # Iterar sobre los archivos en el directorio
  for nombre_archivo in lista_archivos:
      match = patron_nombre_archivo.match(nombre_archivo)
      if match:
          codigo_estacion = match.group(1)
          sustancia = match.group(2)
          ruta_completa_archivo = os.path.join(directorio_estaciones, nombre_archivo)
          if logger:
            print(f"Procesando archivo: {nombre_archivo} (Estación: {codigo_estacion}, Sustancia: {sustancia})")

          try:
              # Leer el archivo JSON
              with open(ruta_completa_archivo, 'r', encoding='utf-8') as f:
                  datos_json = json.load(f)

              # Asumimos que los datos relevantes están bajo "response" y luego "valores"
              valores = datos_json.get("response", {}).get("valores", [])

              if not valores:
                  print(f"ADVERTENCIA: No se encontraron valores en el archivo {nombre_archivo}.")
                  continue

              # Crear un DataFrame temporal para esta sustancia y estación
              df_sustancia = pd.DataFrame(valores)

              # Renombrar la columna 'value_formateado' con el nombre de la sustancia
              if 'value_formateado' in df_sustancia.columns:
                  # Convertir a numérico, coaccionando errores
                  df_sustancia[sustancia] = pd.to_numeric(df_sustancia['value_formateado'], errors='coerce')
                  # Seleccionar las columnas 'Tiempo' y la nueva columna de sustancia, y añadir 'Estación'
                  df_sustancia = df_sustancia[['tiempo', sustancia]].copy() # Usar copy() para evitar SettingWithCopyWarning
                  df_sustancia['Estación'] = codigo_estacion # Añadir la columna 'Estación'

                  # Renombrar 'tiempo' a 'Tiempo' para consistencia
                  df_sustancia.rename(columns={'tiempo': 'Tiempo'}, inplace=True)

              else:
                  print(f"ADVERTENCIA: Columna 'value_formateado' no encontrada en {nombre_archivo}.")
                  continue

              # Fusionar este DataFrame de sustancia con el DataFrame existente para la estación
              if codigo_estacion not in matrices_tiempo_por_estacion:
                  # Si es la primera sustancia para esta estación, inicializar el DataFrame
                  matrices_tiempo_por_estacion[codigo_estacion] = df_sustancia[['Tiempo', 'Estación', sustancia]] # Incluir Estación aquí
              else:
                  # Fusionar sobre las columnas 'Tiempo' y 'Estación'
                  matrices_tiempo_por_estacion[codigo_estacion] = pd.merge(
                      matrices_tiempo_por_estacion[codigo_estacion],
                      df_sustancia[['Tiempo', 'Estación', sustancia]], # Fusionar con Estación y Sustancia
                      on=['Tiempo', 'Estación'],
                      how='outer' # Usar outer join para mantener todos los tiempos y estaciones
                  )

          except FileNotFoundError:
              print(f"ERROR: No se encontró el archivo en la ruta: {ruta_completa_archivo}")
          except json.JSONDecodeError:
              print(f"ERROR: No se pudo decodificar el archivo JSON {ruta_completa_archivo}. Verifica su formato.")
          except Exception as e:
              print(f"Ocurrió un error inesperado al procesar {nombre_archivo}: {e}")

  # Ordenar los DataFrames por 'Tiempo' y 'Estación' y reordenar columnas para mostrar 'Estación' después de 'Tiempo'
  print("\n--- DataFrames de Series de Tiempo por Estación ---")
  for estacion, df_tiempo in matrices_tiempo_por_estacion.items():
      #if view:
      print(f"\nEstación: {estacion}")

      df_tiempo.sort_values(["Estación", "Tiempo"], inplace=True)

      # Reordenar columnas: poner 'Estación' después de 'Tiempo'
      cols = df_tiempo.columns.tolist()
      if 'Tiempo' in cols and 'Estación' in cols:
          cols.remove('Estación')
          cols.insert(cols.index('Tiempo') + 1, 'Estación')
          df_tiempo = df_tiempo[cols]
      if view:
        display(df_tiempo)

  # Ahora 'matrices_tiempo_por_estacion' es un diccionario donde cada clave es el código de la estación
  # y cada valor es un DataFrame con 'Tiempo', 'Estación' y columnas para cada sustancia encontrada para esa estación.

  # Añadir el retorno del diccionario
  return matrices_tiempo_por_estacion

In [2]:
# Biblioteca para dar formato a las tablas en Google Colab
from google.colab import data_table
data_table.enable_dataframe_formatter()
import pandas as kd

# Generación de las tablas con los datos de sustancias de las estaciones
directorio_estaciones = '/content/Estaciones'
kd = generar_tablas_estaciones(directorio_estaciones, view=True, logger=False)
# display(kd.get('C302'))

Output hidden; open in https://colab.research.google.com to view.